In [1]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import os

In [2]:
log_reg_model = pickle.load(open("ml_models/models_weights/logistic_regression/logistic_regression_93tr_91ts.sav", "rb"))
tf_idf_model = pickle.load(open("ml_models/models_weights/tf_idf/tf_idf_vectorizer_50000_fetures.pickle", "rb"))

In [3]:
path_to_read = 'csv_down_files/preprocessed/COVID19-tweetID-2020-01/' # January Month of 2020 
path_to_save = 'csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/' # Classify each file tweets

# Small files functions

In [ ]:
def classify_tweets_of_file(file_path, df_file):  
    tweets = list(df_file['full_text'])
    tweets_features = tf_idf_model.transform(tweets)
    tweets_features_array = tweets_features.toarray()
    predict = log_reg_model.predict(tweets_features_array)
    classifed_dict    = {'tweet_text': tweets, 'class': predict}
    classifed_data    = pd.DataFrame(classifed_dict)
    return classifed_data



def read_direction_to_classify(direction_path, path_to_save):
    try:
        all_path_files                          = os.listdir(direction_path)
        cls = []
        
        for file_path in all_path_files:
            file_path                           = direction_path + file_path
            df_file   = pd.read_csv(file_path, error_bad_lines=False, encoding='utf-8', engine='python')
            print("=========")
#             cls.extend(df_file['class'])
            print(file_path)
            classify_tweets                     = classify_tweets_of_file(file_path, df_file)
            print("--------------")
            file_path                           = file_path.split('/')
            file_name                           = file_path[3]
            path_to_save_excel = path_to_save
            path_to_save_csv = path_to_save + "csv/" + file_name
            print(path_to_save_csv)
#             path_to_save_excel = path_to_save_excel + "excel/" + file_name
#             path_to_save_excel = path_to_save_excel.replace(".csv", '.xlsx')
#             classify_tweets.to_csv(path_to_save_csv, index=False)
#             classify_tweets.to_excel(path_to_save_excel, index=False)
    except Exception as e:
        file = open("f1_scores/direction_and_file_handleing_file.log","+a")
        file.write("This error related to function read_direction_preprocess of direction_and_file_handleing file in direction config_files n"
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return cls

cls = read_direction_to_classify(path_to_read, path_to_save)
len(cls)

# Big files functions

In [ ]:
def big_file_classify(df_file, file_name, path_to_save):
    tweets = list(df_file['full_text'])
    df_len = len(df_file)
    to_index = 0
    prdict_all = []
    while to_index <= df_len:
        part_of_tweets = tweets[to_index: to_index + 5000]
        to_index = to_index + 5000
        tweets_features = tf_idf_model.transform(part_of_tweets)
        tweets_features_array = tweets_features.toarray()
        predict = log_reg_model.predict(tweets_features_array)
        prdict_all.extend(predict)
        
#         print(len(part_of_tweets))
#     print(len(prdict_all))
    classifed_dict    = {'tweet_text': tweets, 'class': prdict_all}
    classifed_data    = pd.DataFrame(classifed_dict)
    path_to_save = path_to_save + file_name
#     path_to_save_csv = path_to_save + "csv/" + file_name
#     file_name = file_name.replace('.csv', '.xlsx')
#     path_to_save_excel = path_to_save + "excel/" + file_name
    classifed_data.to_csv(path_to_save, index=False)
    print(len(classifed_data))
    print(len(tweets))
    print(len(prdict_all))
    print("="*50)
#     classifed_data.to_excel(path_to_save_excel, index=False)
    return True

In [ ]:
def read_direction_big_files(direction_path, path_to_save):
    try:
        all_path_files                          = os.listdir(direction_path)
        for file_path in all_path_files:
            file_path                           = direction_path + file_path
            df_file   = pd.read_csv(file_path, error_bad_lines=False, encoding='utf-8', engine='python')
            
            file_path                           = file_path.split('/')
            file_name                           = file_path[3]
            big_file_classify(df_file, file_name, path_to_save)
    except Exception as e:
        file = open("logs/direction_and_file_handleing_file.log","+a")
        file.write("This error related to function read_direction_preprocess of direction_and_file_handleing file in direction config_files n"
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return True

In [ ]:
big_files = 'csv_down_files/preprocessed/bigfiles/'
path_to_save = 'csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/'
read_direction_big_files(big_files, path_to_save)

In [ ]:
def get_all_tweets(direction_read):
    try:
#         total = 0
        all_path_files              = os.listdir(direction_read)
        all_files_tweets            = []
        all_files_tweets_predicited = []
        for file_path in all_path_files:
            file_name = file_path
            file_path           = direction_read + file_path
            df_file             = pd.read_csv(file_path, lineterminator='\n', error_bad_lines=False, encoding='utf-8')
            print(file_path)
            print(len(df_file))
            tweets_features = tf_idf_model.transform(list(df_file['full_text']))
            tweets_features_array = tweets_features.toarray()
            predict = log_reg_model.predict(tweets_features_array)
            
            all_files_tweets    += list(df_file['full_text'])
            all_files_tweets_predicited    += list(predict)
            
            path_to_save = 'csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/'
            path_to_save = path_to_save + file_name
            dicts_df                = {"tweet_text": list(df_file['full_text']), 'class': list(predict)}
            dicts_df                 = pd.DataFrame(dicts_df)
            dicts_df.to_csv(path_to_save, index=False)
#             print("===========")
#             all_files_tweets_predicited    += list(df_file['class'])
#             print(len(all_files_tweets))
        df_file                = {"tweet_text": all_files_tweets, 'class': all_files_tweets_predicited}
        df_file                 = pd.DataFrame(dicts_df)
              
    except Exception as e:
        file = open("logs/get_tweet_to_classify.log","+a")
        file.write("This error related to function get_random_number_of_tweets of get_tweet_to_classify file in direction config_files n"
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return df_file


In [ ]:
df_file = get_all_tweets(path_to_read)
df_file.head()

In [ ]:
df = pd.read_csv('csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-03.csv')

In [ ]:
tweets = list(df_file['tweet_text'])
tweets_features = tf_idf_model.transform(tweets)
tweets_features_array = tweets_features.toarray()
predict = log_reg_model.predict(tweets_features_array)

classifed_dict    = {'tweet_text': tweets, 'class': predict}

classifed_data    = pd.DataFrame(classifed_dict)
classifed_data.head()

In [ ]:
path = path_to_save + 'csv/'
df_file = get_all_tweets(path)

In [ ]:
df_file.head()

In [ ]:
import numpy as np
cls = np.array(cls)
cls[:10]

In [ ]:
class_1 = np.sum(cls == 1)
class_1

In [ ]:
class_0 = np.sum(cls == 0)
class_0

In [ ]:
print("Number of coronavirus Tweets: ", class_1)
print("Number of other Tweets: ", class_0)

In [ ]:
histogram = [0, 1]

labels= 'Class 1 Coronavirus', 'Class 0 General'
sizes = [class_1, class_0]
explode= (0, 0.1)  
fig1, ax1= plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title("Coronavirus Vs General Tweets")
plt.savefig('test2')

In [46]:
def one_file_for_all_tweets(direction_path):
    try:
        all_path_files                          = os.listdir(direction_path)
        tweets = []
        classes = []
        for file_path in all_path_files:
            file_path                           = direction_path + file_path
            df_file   = pd.read_csv(file_path, error_bad_lines=False, encoding='utf-8', engine='python')
            tweets += list(df_file['tweet_text'])
            classes += list(df_file['class'])
        
        
        all_classifed_tweets    = {'tweet_text': tweets, 'class': classes}
        all_classifed_tweets = pd.DataFrame(all_classifed_tweets)
        all_classifed_tweets = all_classifed_tweets.dropna()
        pass_to_save = "csv_down_files/"
        all_classifed_tweets.to_csv(pass_to_save + "all_classifed_tweets_304418.csv", index=False)
        all_classifed_tweets.to_excel(pass_to_save + "all_classifed_tweets_304418.xlsx", index=False)
    except Exception as e:
        print('======')
        file = open("logs/direction_and_file_handleing_file.log","+a")
        file.write("This error related to function read_direction_preprocess of direction_and_file_handleing file in direction config_files n"
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return True

In [47]:
reading_path = 'csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/'
one_file_for_all_tweets(reading_path)

True

In [48]:
df = pd.read_csv('csv_down_files/all_classifed_tweets_304418.csv')
len(df)

304415

In [49]:
df.head()

,tweet_text,class
0,منظمه الصحه العالميه تدعو الصين التحقيق في مص...,1.0
1,الصحه العالميه ظهور كورونا جديد في الصين,1.0
2,الصحه العالميه ظهور كورونا جديد في الصين\...,1.0
3,وضع الاطفال الشاشات صباحا يزيد خطر تعرضهم لمشك...,0.0
4,الصحه العالميه تحذر ظهور كورونا جديد في الص...,1.0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304415 entries, 0 to 304414
Data columns (total 2 columns):
tweet_text    304415 non-null object
class         304415 non-null float64
dtypes: float64(1), object(1)
memory usage: 4.6+ MB


In [51]:
dfs = df.astype({"tweet_text": object, "class": int})

In [52]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304415 entries, 0 to 304414
Data columns (total 2 columns):
tweet_text    304415 non-null object
class         304415 non-null int64
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [53]:
dfs.to_csv('csv_down_files/all_classifed_tweets_304418.csv', index=False)

In [54]:
dfs = pd.read_csv('csv_down_files/all_classifed_tweets_304418.csv')
len(dfs)

304415

In [55]:
dfs.head()

,tweet_text,class
0,منظمه الصحه العالميه تدعو الصين التحقيق في مص...,1
1,الصحه العالميه ظهور كورونا جديد في الصين,1
2,الصحه العالميه ظهور كورونا جديد في الصين\...,1
3,وضع الاطفال الشاشات صباحا يزيد خطر تعرضهم لمشك...,0
4,الصحه العالميه تحذر ظهور كورونا جديد في الص...,1


In [56]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304415 entries, 0 to 304414
Data columns (total 2 columns):
tweet_text    304415 non-null object
class         304415 non-null int64
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [57]:
dfs = pd.read_excel('csv_down_files/all_classifed_tweets_304418.xlsx')
dfs.head()

,tweet_text,class
0,منظمه الصحه العالميه تدعو الصين التحقيق في مص...,1
1,الصحه العالميه ظهور كورونا جديد في الصين,1
2,الصحه العالميه ظهور كورونا جديد في الصين\...,1
3,وضع الاطفال الشاشات صباحا يزيد خطر تعرضهم لمشك...,0
4,الصحه العالميه تحذر ظهور كورونا جديد في الص...,1


In [58]:
len(dfs)

304415